# Imports

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import pandas as pd
import time
import pickle

# Initialization of data and downloading info for classic pools

In [2]:
with open('classic_pools_mevs', 'rb') as f: mevs_transactions_list = pickle.load(f)

In [3]:
mevs_transactions_list[0]

'0x8668f06dde23a8ccc0f2c2a594c3882e51bad15a1b7fa4a9dad683e2414d9146'

In [4]:
url = 'https://etherscan.io/tx/'
txds = mevs_transactions_list
driver = webdriver.Chrome(os.getcwd() + "\chromedriver.exe")
df = pd.DataFrame(columns=['txd', 'block', 'block_position', 
                        #    'tx_type', 'base_gas_fee_eth', 'max_gas_fee_eth', 'max_priority_gas_fee_eth', 
                           'gas_usage_by_tx', 'ether_price', 'tx_fee'])

iteration = 0
total_iterations = len(mevs_transactions_list)
for txd in txds:
    driver.get(url + txd)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    block = soup.findAll("div", attrs={'class': 'col-md-9'})[2].findAll('a')[0].text if len(soup.findAll("div", attrs={'class': 'col-md-9'})) != 0 else 0
    transaction_fee = soup.findAll('span', attrs={'id': 'ContentPlaceHolder1_spanGasPrice'})[0].text.split('(')[0].replace('\n', '').replace(' Ether ', '')
    ether_price = soup.findAll('span', attrs={'id': 'ContentPlaceHolder1_spanClosingPrice'})[0].text.replace('\n$', '').replace(' / ETH', '')
    gas_usage_by_tx = soup.findAll('span', attrs={'data-original-title': 'The amount of GAS used by this specific transaction alone'})[0].text.split('(')[0].replace('\n', '').replace(' ', '')
    
   #  driver.find_element(By.XPATH, '/html/body/div[1]/main/div[3]/div[1]/div[2]/div[1]/div/div[14]/a/span').click()
    
    # base_gas_fee_eth = soup.findAll('span', attrs={'class': 'mr-1 mb-1'})[0].get('data-original-title').replace(' Ether', '')
    # max_gas_fee_eth = soup.findAll('span', attrs={'class': 'mr-1 mb-1'})[1].get('data-original-title').replace(' Ether', '')
    # max_priority_gas_fee_eth = soup.findAll('span', attrs={'class': 'mr-1 mb-1'})[2].get('data-original-title').replace(' Ether', '')
    # transaction_type = soup.findAll('span', attrs={'class': 'text-dark'})[4].text
    blocks = soup.findAll('span', attrs={'class': 'text-dark'})
    block_position = blocks[len(blocks) - 1].text
    
    new_row = {'txd': txd,
           'block': block,
           'block_position': block_position,
        #    'tx_type': transaction_type,
        #    'base_gas_fee_eth': base_gas_fee_eth,
        #    'max_gas_fee_eth': max_gas_fee_eth,
        #    'max_priority_gas_fee_eth': max_priority_gas_fee_eth,
           'gas_usage_by_tx': gas_usage_by_tx,
           'ether_price': ether_price,
           'tx_fee': transaction_fee}

    df = df.append(new_row, ignore_index=True)
    print(fr"iteration {iteration} out of {total_iterations}")
    iteration += 1
    time.sleep(5)
    
df

<ipython-input-4-e126b5e500bb>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.getcwd() + "\chromedriver.exe")


,txd,block,block_position,gas_usage_by_tx,ether_price,tx_fee
0,0x8668f06dde23a8ccc0f2c2a594c3882e51bad15a1b7f...,11279366,54,"107,326",478.76,0.000000122000000001
1,0xbffa11c58ec26affd6c477388d70114b287c5dc13381...,11279366,62,"101,057",478.76,0.000000115500000001
2,0x668f2a34df8542f7555ace6b11097659ca0f51449766...,11792953,75,"66,799","1,721.48",0.0000002431
3,0x9e7704076894c0322297eab36eb06dda5e3cc5302549...,11792953,3,"103,505","1,721.48",0.000000301463842552
4,0x68bd6b0b089f9be253a7b47a8bf79889a6b27f68f11b...,11803970,67,"139,130","1,679.14",0.000000140400000001
...,...,...,...,...,...,...
705,0x6359588d8b7bf8851d6da7de3f092d6f88f30a54893d...,13969479,1,"87,023","3,150.83",0.000000096534076405
706,0x0da0019eabf40846f0a0fc8c7a2f53312984111754c2...,13969481,2,"88,383","3,150.83",0.000000109600853456
707,0x8d86190d1027eba21b077940f1c414e3ca06ace37839...,13969481,4,"78,261","3,150.83",0.000000117286586847
708,0x0da2303947534ae2992d7306e4856859020137e50186...,14049774,0,"88,481","2,569.59",0.000000135879365521


In [5]:
df = df.to_pickle('classic_pools_mevs_with_gas_df.pkl')

# Download for STO

In [9]:
with open('sto_pools_mevs', 'rb') as f: mevs_transactions_list = pickle.load(f)

In [11]:
url = 'https://etherscan.io/tx/'
txds = mevs_transactions_list
driver = webdriver.Chrome(os.getcwd() + "\chromedriver.exe")
df = pd.DataFrame(columns=['txd', 'block', 'block_position', 
                        #    'tx_type', 'base_gas_fee_eth', 'max_gas_fee_eth', 'max_priority_gas_fee_eth', 
                           'gas_usage_by_tx', 'ether_price', 'tx_fee'])
iteration = 0
total_iterations = len(mevs_transactions_list)
for txd in txds:
    driver.get(url + txd)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    block = soup.findAll(
        "div", attrs={'class': 'col-md-9'}
    )[2].findAll('a')[0].text if len(soup.findAll("div", attrs={'class': 'col-md-9'})) != 0 else 0
    
    transaction_fee = soup.findAll('span', attrs={
        'id': 'ContentPlaceHolder1_spanGasPrice'}
    )[0].text.split('(')[0].replace('\n', '').replace(' Ether ', '')
    
    ether_price = soup.findAll(
        'span', attrs={'id': 'ContentPlaceHolder1_spanClosingPrice'}
    )[0].text.replace('\n$', '').replace(' / ETH', '')
    
    gas_usage_by_tx = soup.findAll('span', attrs={
        'data-original-title': 'The amount of GAS used by this specific transaction alone'}
    )[0].text.split('(')[0].replace('\n', '').replace(' ', '')
    
    blocks = soup.findAll('span', attrs={'class': 'text-dark'})
    block_position = blocks[len(blocks) - 1].text
    
    new_row = {'txd': txd,
           'block': block,
           'block_position': block_position,
           'gas_usage_by_tx': gas_usage_by_tx,
           'ether_price': ether_price,
           'tx_fee': transaction_fee}

    df = df.append(new_row, ignore_index=True)
    print(fr"iteration {iteration} out of {total_iterations}")
    iteration += 1
    time.sleep(5)
    
df

<ipython-input-11-1258197325ac>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.getcwd() + "\chromedriver.exe")


iteration 0 out of 132
iteration 1 out of 132
iteration 2 out of 132
iteration 3 out of 132
iteration 4 out of 132
iteration 5 out of 132
iteration 6 out of 132
iteration 7 out of 132
iteration 8 out of 132
iteration 9 out of 132
iteration 10 out of 132
iteration 11 out of 132
iteration 12 out of 132
iteration 13 out of 132
iteration 14 out of 132
iteration 15 out of 132
iteration 16 out of 132
iteration 17 out of 132
iteration 18 out of 132
iteration 19 out of 132
iteration 20 out of 132
iteration 21 out of 132
iteration 22 out of 132
iteration 23 out of 132
iteration 24 out of 132
iteration 25 out of 132
iteration 26 out of 132
iteration 27 out of 132
iteration 28 out of 132
iteration 29 out of 132
iteration 30 out of 132
iteration 31 out of 132
iteration 32 out of 132
iteration 33 out of 132
iteration 34 out of 132
iteration 35 out of 132
iteration 36 out of 132
iteration 37 out of 132
iteration 38 out of 132
iteration 39 out of 132
iteration 40 out of 132
iteration 41 out of 132
it

,txd,block,block_position,gas_usage_by_tx,ether_price,tx_fee
0,0x23c2723eeb5a6186ec39c9f34b279919e941f9fb53f4...,12141124,3,"103,352","1,841.40",0.000000001
1,0x52613b563e64f3b6dd4f57f8a9b541e383717ca68e50...,12141124,0,"126,727","1,841.40",0.000000001
2,0x230718cd1083dc56ab1a18c344c672d60de018c943db...,12552368,2,"76,866","2,706.52",0
3,0xad4e7af1f274bb163f8a1b0f6855acbde840ec1c96b3...,12552368,0,"109,491","2,706.52",0
4,0xab522b2083fc2dd28262d2410642d527fb0986076f2f...,12662859,3,"110,550","2,166.61",0
...,...,...,...,...,...,...
127,0xd47fbdbbfaf30ea607c7ef83c3da8ce39457909e4dac...,13067645,4,"123,126","3,226.59",0.00000003476130489
128,0x3fd5d614c6fa5176bcc30f734eafd486ce042143cbea...,13086996,2,"133,773","3,172.57",0.000000047930125437
129,0xd29a3f30fc769bf642e0e89b903bb10c36642f8c7fa5...,13086996,0,"159,961","3,172.57",0.000000047930125437
130,0x114bfeeb481692518959ccd62933b3d890a38f97506e...,13250059,3,"180,198","3,435.76",0.00000004160346301


In [12]:
df = df.to_pickle('sto_pools_mevs_with_gas_df.pkl')

# Download for meme pools

In [13]:
with open('meme_pools_mevs', 'rb') as f: mevs_transactions_list = pickle.load(f)

In [15]:
url = 'https://etherscan.io/tx/'
txds = mevs_transactions_list
driver = webdriver.Chrome(os.getcwd() + "\chromedriver.exe")
df = pd.DataFrame(columns=['txd', 'block', 'block_position', 
                        #    'tx_type', 'base_gas_fee_eth', 'max_gas_fee_eth', 'max_priority_gas_fee_eth', 
                           'gas_usage_by_tx', 'ether_price', 'tx_fee'])
iteration = 0
total_iterations = len(mevs_transactions_list)

for txd in txds:
    driver.get(url + txd)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    block = soup.findAll(
        "div", attrs={'class': 'col-md-9'}
    )[2].findAll('a')[0].text if len(soup.findAll("div", attrs={'class': 'col-md-9'})) != 0 else 0
    
    transaction_fee = soup.findAll(
        'span', attrs={'id': 'ContentPlaceHolder1_spanGasPrice'}
    )[0].text.split('(')[0].replace('\n', '').replace(' Ether ', '')
    
    ether_price = soup.findAll(
        'span', attrs={'id': 'ContentPlaceHolder1_spanClosingPrice'}
    )[0].text.replace('\n$', '').replace(' / ETH', '')
    
    gas_usage_by_tx = soup.findAll(
        'span', attrs={'data-original-title': 'The amount of GAS used by this specific transaction alone'}
    )[0].text.split('(')[0].replace('\n', '').replace(' ', '')
    
    blocks = soup.findAll('span', attrs={'class': 'text-dark'})
    block_position = blocks[len(blocks) - 1].text
    
    new_row = {'txd': txd,
           'block': block,
           'block_position': block_position,
           'gas_usage_by_tx': gas_usage_by_tx,
           'ether_price': ether_price,
           'tx_fee': transaction_fee}

    df = df.append(new_row, ignore_index=True)
    print(fr"iteration {iteration} out of {total_iterations}")
    iteration += 1
    time.sleep(5)
    
df

<ipython-input-15-60fa464f5e2f>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.getcwd() + "\chromedriver.exe")


iteration 0 out of 24483
iteration 1 out of 24483
iteration 2 out of 24483
iteration 3 out of 24483
iteration 4 out of 24483
iteration 5 out of 24483
iteration 6 out of 24483
iteration 7 out of 24483
iteration 8 out of 24483
iteration 9 out of 24483
iteration 10 out of 24483
iteration 11 out of 24483
iteration 12 out of 24483
iteration 13 out of 24483
iteration 14 out of 24483
iteration 15 out of 24483
iteration 16 out of 24483
iteration 17 out of 24483
iteration 18 out of 24483
iteration 19 out of 24483
iteration 20 out of 24483
iteration 21 out of 24483
iteration 22 out of 24483
iteration 23 out of 24483
iteration 24 out of 24483
iteration 25 out of 24483
iteration 26 out of 24483
iteration 27 out of 24483
iteration 28 out of 24483
iteration 29 out of 24483
iteration 30 out of 24483
iteration 31 out of 24483
iteration 32 out of 24483
iteration 33 out of 24483
iteration 34 out of 24483
iteration 35 out of 24483
iteration 36 out of 24483
iteration 37 out of 24483
iteration 38 out of 24

iteration 308 out of 24483
iteration 309 out of 24483
iteration 310 out of 24483
iteration 311 out of 24483
iteration 312 out of 24483
iteration 313 out of 24483
iteration 314 out of 24483
iteration 315 out of 24483
iteration 316 out of 24483
iteration 317 out of 24483
iteration 318 out of 24483
iteration 319 out of 24483
iteration 320 out of 24483
iteration 321 out of 24483
iteration 322 out of 24483
iteration 323 out of 24483
iteration 324 out of 24483
iteration 325 out of 24483
iteration 326 out of 24483
iteration 327 out of 24483
iteration 328 out of 24483
iteration 329 out of 24483
iteration 330 out of 24483
iteration 331 out of 24483
iteration 332 out of 24483
iteration 333 out of 24483
iteration 334 out of 24483
iteration 335 out of 24483
iteration 336 out of 24483
iteration 337 out of 24483
iteration 338 out of 24483
iteration 339 out of 24483
iteration 340 out of 24483
iteration 341 out of 24483
iteration 342 out of 24483
iteration 343 out of 24483
iteration 344 out of 24483
i

iteration 612 out of 24483
iteration 613 out of 24483
iteration 614 out of 24483
iteration 615 out of 24483
iteration 616 out of 24483
iteration 617 out of 24483
iteration 618 out of 24483
iteration 619 out of 24483
iteration 620 out of 24483
iteration 621 out of 24483
iteration 622 out of 24483
iteration 623 out of 24483
iteration 624 out of 24483
iteration 625 out of 24483
iteration 626 out of 24483
iteration 627 out of 24483
iteration 628 out of 24483
iteration 629 out of 24483
iteration 630 out of 24483
iteration 631 out of 24483
iteration 632 out of 24483
iteration 633 out of 24483
iteration 634 out of 24483
iteration 635 out of 24483
iteration 636 out of 24483
iteration 637 out of 24483
iteration 638 out of 24483
iteration 639 out of 24483
iteration 640 out of 24483
iteration 641 out of 24483
iteration 642 out of 24483
iteration 643 out of 24483
iteration 644 out of 24483
iteration 645 out of 24483
iteration 646 out of 24483
iteration 647 out of 24483
iteration 648 out of 24483
i

KeyboardInterrupt: 

In [ ]:
df = df.to_pickle('meme_pools_mevs_with_gas_df.pkl')

# Download for NFT pools

In [ ]:
with open('nft_pools_mevs', 'rb') as f: mevs_transactions_list = pickle.load(f)

In [ ]:
url = 'https://etherscan.io/tx/'
txds = mevs_transactions_list
driver = webdriver.Chrome(os.getcwd() + "\chromedriver.exe")
df = pd.DataFrame(columns=['txd', 'block', 'block_position', 
                        #    'tx_type', 'base_gas_fee_eth', 'max_gas_fee_eth', 'max_priority_gas_fee_eth', 
                           'gas_usage_by_tx', 'ether_price', 'tx_fee'])

iteration = 0
total_iterations = len(mevs_transaction_list)
for txd in txds:
    driver.get(url + txd)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    block = soup.findAll(
       "div", attrs={'class': 'col-md-9'}
    )[2].findAll('a')[0].text if len(soup.findAll("div", attrs={'class': 'col-md-9'})) != 0 else 0
   
    transaction_fee = soup.findAll(
       'span', attrs={'id': 'ContentPlaceHolder1_spanGasPrice'}
    )[0].text.split('(')[0].replace('\n', '').replace(' Ether ', '')
   
    ether_price = soup.findAll(
       'span', attrs={'id': 'ContentPlaceHolder1_spanClosingPrice'}
    )[0].text.replace('\n$', '').replace(' / ETH', '')
   
    gas_usage_by_tx = soup.findAll(
       'span', attrs={'data-original-title': 'The amount of GAS used by this specific transaction alone'}
    )[0].text.split('(')[0].replace('\n', '').replace(' ', '')
    
    blocks = soup.findAll('span', attrs={'class': 'text-dark'})
    block_position = blocks[len(blocks) - 1].text
    
    new_row = {'txd': txd,
             'block': block,
             'block_position': block_position,
             'gas_usage_by_tx': gas_usage_by_tx,
             'ether_price': ether_price,
             'tx_fee': transaction_fee}

    df = df.append(new_row, ignore_index=True)
    print(fr"iteration {iteration} out of {total_iterations}")
    iteration += 1
    time.sleep(5)
    
df

In [ ]:
df = df.to_pickle('nft_pools_mevs_with_gas_df.pkl')

# Collecting possible MEVs

## Classic tokens possible MEVs

In [26]:
with open('classic_pools_possible_mevs', 'rb') as f: classic_possible_mevs_transactions_list = pickle.load(f)

In [50]:
def collect_gas_info(mevs_transactions_list: list, interval_between_iterations: float = 5.0, name_of_file: str='df') -> pd.DataFrame:
    url = 'https://etherscan.io/tx/'
    txds = mevs_transactions_list
    driver = webdriver.Chrome(os.getcwd() + "\chromedriver.exe")
    df = pd.DataFrame(columns=['txd', 'block', 'block_position',
                               #    'tx_type', 'base_gas_fee_eth', 'max_gas_fee_eth', 'max_priority_gas_fee_eth',
                               'gas_usage_by_tx', 'ether_price', 'tx_fee'])
    iteration = 0
    total_iterations = len(mevs_transactions_list)

    for txd in txds:
        driver.get(url + txd)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        possible_block = soup.findAll("div", attrs={'class': 'col-md-9'})
        possible_transaction_fee = soup.findAll('span', attrs={'id': 'ContentPlaceHolder1_spanGasPrice'})
        possible_ether_price = soup.findAll('span', attrs={'id': 'ContentPlaceHolder1_spanClosingPrice'})
        possible_gas_tx_usage = soup.findAll('span', attrs={'data-original-title': 'The amount of GAS used by this specific transaction alone'})
        possible_blocks = soup.findAll('span', attrs={'class': 'text-dark'})

        block = possible_block[2].findAll('a')[0].text if len(possible_block) != 0 else 0
        transaction_fee = possible_transaction_fee[0].text.split('(')[0].replace('\n', '').replace(' Ether ', '') \
            if len(possible_transaction_fee) != 0 else 0
        ether_price = possible_ether_price[0].text.replace('\n$', '').replace(' / ETH', '') \
            if len(possible_ether_price) != 0 else 0
        gas_usage_by_tx = possible_gas_tx_usage[0].text.split('(')[0].replace('\n', '').replace(' ', '') \
            if len(possible_gas_tx_usage) != 0 else 0
        blocks = possible_blocks
        block_position = blocks[len(blocks) - 1].text

        new_row = {'txd': txd,
                   'block': block,
                   'block_position': block_position,
                   'gas_usage_by_tx': gas_usage_by_tx,
                   'ether_price': ether_price,
                   'tx_fee': transaction_fee}
        df = df.append(new_row, ignore_index=True)
        print(fr"iteration {iteration} out of {total_iterations}")
        iteration += 1
        if iteration % 1000 == 0:
            df.to_csv(name_of_file + '_' + str(iteration))
        time.sleep(interval_between_iterations)

    return df

In [29]:
classic_gas_df = collect_gas_info(classic_possible_mevs_transactions_list, 4)

<ipython-input-27-6f47ae0f9fb8>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.getcwd() + "\chromedriver.exe")


iteration 0 out of 18323
iteration 1 out of 18323
iteration 2 out of 18323
iteration 3 out of 18323
iteration 4 out of 18323
iteration 5 out of 18323
iteration 6 out of 18323
iteration 7 out of 18323
iteration 8 out of 18323
iteration 9 out of 18323
iteration 10 out of 18323
iteration 11 out of 18323
iteration 12 out of 18323
iteration 13 out of 18323
iteration 14 out of 18323
iteration 15 out of 18323
iteration 16 out of 18323
iteration 17 out of 18323
iteration 18 out of 18323
iteration 19 out of 18323
iteration 20 out of 18323
iteration 21 out of 18323
iteration 22 out of 18323
iteration 23 out of 18323
iteration 24 out of 18323
iteration 25 out of 18323
iteration 26 out of 18323
iteration 27 out of 18323
iteration 28 out of 18323
iteration 29 out of 18323
iteration 30 out of 18323
iteration 31 out of 18323
iteration 32 out of 18323
iteration 33 out of 18323
iteration 34 out of 18323
iteration 35 out of 18323
iteration 36 out of 18323
iteration 37 out of 18323
iteration 38 out of 18

In [30]:
classic_gas_df = classic_gas_df.to_pickle('classic_pools_possible_mevs_with_gas_df.pkl')

In [ ]:
with open('sto_pools_possible_mevs', 'rb') as f: sto_possible_mevs_transactions_list = pickle.load(f)

In [ ]:
sto_gas_df = collect_gas_info(sto_possible_mevs_transactions_list, 4)

In [ ]:
sto_gas_df = sto_gas_df.to_pickle('sto_pools_possible_mevs_with_gas_df.pkl')

# NFT tokens possible MEVs

In [31]:
with open('nft_pools_possible_mevs', 'rb') as f: nft_possible_mevs_transactions_list = pickle.load(f)

In [32]:
nft_gas_df = collect_gas_info(nft_possible_mevs_transactions_list, 4)

<ipython-input-27-6f47ae0f9fb8>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.getcwd() + "\chromedriver.exe")


iteration 0 out of 4244
iteration 1 out of 4244
iteration 2 out of 4244
iteration 3 out of 4244
iteration 4 out of 4244
iteration 5 out of 4244
iteration 6 out of 4244
iteration 7 out of 4244
iteration 8 out of 4244
iteration 9 out of 4244
iteration 10 out of 4244
iteration 11 out of 4244
iteration 12 out of 4244
iteration 13 out of 4244
iteration 14 out of 4244
iteration 15 out of 4244
iteration 16 out of 4244
iteration 17 out of 4244
iteration 18 out of 4244
iteration 19 out of 4244
iteration 20 out of 4244
iteration 21 out of 4244
iteration 22 out of 4244
iteration 23 out of 4244
iteration 24 out of 4244
iteration 25 out of 4244
iteration 26 out of 4244
iteration 27 out of 4244
iteration 28 out of 4244
iteration 29 out of 4244
iteration 30 out of 4244
iteration 31 out of 4244
iteration 32 out of 4244
iteration 33 out of 4244
iteration 34 out of 4244
iteration 35 out of 4244
iteration 36 out of 4244
iteration 37 out of 4244
iteration 38 out of 4244
iteration 39 out of 4244
iteration 

In [33]:
nft_gas_df = nft_gas_df.to_pickle('nft_pools_possible_mevs_with_gas_df.pkl')

# Meme tokens possible MEVs

In [38]:
with open('meme_pools_possible_mevs', 'rb') as f: meme_possible_mevs_transactions_list = pickle.load(f)

In [51]:
meme_gas_df = collect_gas_info(meme_possible_mevs_transactions_list, 3.0, 'meme_pools_possible_mevs_with_gas_df')

<ipython-input-50-2938e6d55e2e>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.getcwd() + "\chromedriver.exe")


iteration 0 out of 37259
iteration 1 out of 37259
iteration 2 out of 37259
iteration 3 out of 37259
iteration 4 out of 37259
iteration 5 out of 37259
iteration 6 out of 37259
iteration 7 out of 37259
iteration 8 out of 37259
iteration 9 out of 37259
iteration 10 out of 37259
iteration 11 out of 37259
iteration 12 out of 37259
iteration 13 out of 37259
iteration 14 out of 37259
iteration 15 out of 37259
iteration 16 out of 37259
iteration 17 out of 37259
iteration 18 out of 37259
iteration 19 out of 37259
iteration 20 out of 37259
iteration 21 out of 37259
iteration 22 out of 37259
iteration 23 out of 37259
iteration 24 out of 37259
iteration 25 out of 37259
iteration 26 out of 37259
iteration 27 out of 37259
iteration 28 out of 37259
iteration 29 out of 37259
iteration 30 out of 37259
iteration 31 out of 37259
iteration 32 out of 37259
iteration 33 out of 37259
iteration 34 out of 37259
iteration 35 out of 37259
iteration 36 out of 37259
iteration 37 out of 37259
iteration 38 out of 37

KeyboardInterrupt: 

In [ ]:
meme_gas_df = meme_gas_df.to_pickle('meme_pools_possible_mevs_with_gas_df.pkl')